In [1]:
import pandas as pd

In [3]:
df_channel = pd.read_csv('orders_channels.csv')
df_country = pd.read_csv('orders_country.csv')
df_tickets = pd.read_csv('orders_tickets.csv')

In [4]:
df_channel.head()

,id,date,channel_id
0,10173,2017-06-12,39.0
1,95062,2017-09-11,35.0
2,171081,2017-07-05,39.0
3,122867,2017-08-18,39.0
4,107186,2017-11-23,NaN


In [5]:
df_channel.shape

(2054059, 3)

In [6]:
df_country.head()

,id,country_1,country_2
0,10503,24.0,NaN
1,79360,24.0,0
2,81388,24.0,15
3,82828,24.0,9
4,38756,23.0,NaN


In [9]:
df_country.shape

(1917668, 3)

In [7]:
df_tickets.head()

,id,n_tickets,type
0,1586391,2,pax
1,438232,2,pax
2,270896,1,pax
3,1181593,2,pax
4,964842,1,pax


In [8]:
df_tickets.shape

(1999716, 3)

In [10]:
df_merged = pd.merge(df_channel, df_tickets, how='left', on=['id'])

In [11]:
df_merged = pd.merge(df_merged, df_country, how='left', on=['id'])

In [12]:
df_merged.head()

,id,date,channel_id,n_tickets,type,country_1,country_2
0,10173,2017-06-12,39.0,2,pax,24.0,NaN
1,95062,2017-09-11,35.0,1,pax,12.0,NaN
2,171081,2017-07-05,39.0,1,pax,4.0,11
3,122867,2017-08-18,39.0,1,other,24.0,17
4,107186,2017-11-23,NaN,2,pax,12.0,20


In [13]:
df_merged.shape

(2084458, 7)

In [14]:
len(df_merged.id.tolist())

2084458

In [15]:
len(set(df_merged.id))

1970544

### some orders are assigned to two countries, with country_1 being more important. if there is no value in "country_1", use "country_2"

In [16]:
def f(x, y):
    if x is None:
        return y
    return x
               
df_merged['country'] = df_merged['country_1'].apply(f, args=[df_merged['country_2']])

In [17]:
df_merged.head()

,id,date,channel_id,n_tickets,type,country_1,country_2,country
0,10173,2017-06-12,39.0,2,pax,24.0,NaN,24.0
1,95062,2017-09-11,35.0,1,pax,12.0,NaN,12.0
2,171081,2017-07-05,39.0,1,pax,4.0,11,4.0
3,122867,2017-08-18,39.0,1,other,24.0,17,24.0
4,107186,2017-11-23,NaN,2,pax,12.0,20,12.0


In [24]:
len(df_merged.id)

2084458

In [25]:
len(set(df_merged.id))

1970544

In [18]:
df_merged_grouped = df_merged.groupby(['date', 'channel_id', 'country'])

In [23]:
len(df_merged_grouped.groups)

409587